<a href="https://colab.research.google.com/github/vkukkapalli1/HDS_5210_vsk/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This project analyzes **football team performance** across multiple leagues:
- **Premier League (EPL)**,
- **La Liga**,
- **Serie A**,  
- **Bundesliga**, and
- **Ligue 1**

Key metrics include:
- **Goal conversion rate**
- **Expected goals (xG)**
- **Half-time performance metrics** (win and comeback percentages)
- **Defensive performance**
- **League competitiveness**
- **Referee impact**



This notebook is structured to cater to:
- **Coaches**: Insights into team strategies and areas for improvement.
- **Bettors**: Identifying patterns for informed betting.
- **Analysts**: Advanced metrics for performance evaluation.
- **Fans**: Simple comparisons between top clubs in Europe.

### Project Goals
1. **Compare teams across leagues** in terms of attacking and defensive performance.
2. **Analyze referee influence** on match outcomes.
3. **Examine league competitiveness** using statistical metrics.
4. **Provide actionable insights** for coaches, bettors, and fans.


In [ ]:
# I'm importing all the libraries I think are required for my project.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
epl = pd.read_csv('/content/england-premier-league-2019-to-2020.csv')
laliga = pd.read_csv('/content/spain-la-liga-primera-division-2019-to-2020.csv')
bundesliga = pd.read_csv('/content/germany-bundesliga-1-2019-to-2020.csv')
serie_a = pd.read_csv('/content/italy-serie-a-2019-to-2020.csv')
ligue_1 = pd.read_csv('/content/france-le-championnat-2019-to-2020.csv')

eng_clubs_raw = pd.read_json('https://raw.githubusercontent.com/openfootball/football.json/refs/heads/master/2019-20/en.1.clubs.json')
spa_clubs_raw = pd.read_json('https://raw.githubusercontent.com/openfootball/football.json/refs/heads/master/2019-20/es.1.clubs.json')
ger_clubs_raw = pd.read_json('https://raw.githubusercontent.com/openfootball/football.json/refs/heads/master/2019-20/de.1.clubs.json')
ita_clubs_raw = pd.read_json('https://raw.githubusercontent.com/openfootball/football.json/refs/heads/master/2019-20/it.1.clubs.json')
fre_clubs_raw = pd.read_json('https://raw.githubusercontent.com/openfootball/football.json/refs/heads/master/2019-20/fr.1.clubs.json')


In [ ]:
eng_clubs_raw.head()
#as we can see the dataframe here is not normalized, so we need to normalize for us to use the data from JSONs
#this is same for all raw club data.

,name,clubs
0,Premier League 2019/20,"{'name': 'Watford FC', 'code': 'WAT', 'country..."
1,Premier League 2019/20,"{'name': 'AFC Bournemouth', 'code': 'BOU', 'co..."
2,Premier League 2019/20,"{'name': 'Norwich City FC', 'code': 'NOR', 'co..."
3,Premier League 2019/20,"{'name': 'Burnley FC', 'code': 'BUR', 'country..."
4,Premier League 2019/20,"{'name': 'Manchester United FC', 'code': 'MUN'..."


In [ ]:
# Normalize the 'clubs' field for all leagues
eng_clubs = pd.json_normalize(eng_clubs_raw['clubs'])
spa_clubs = pd.json_normalize(spa_clubs_raw['clubs'])
ger_clubs = pd.json_normalize(ger_clubs_raw['clubs'])
ita_clubs = pd.json_normalize(ita_clubs_raw['clubs'])
fre_clubs = pd.json_normalize(fre_clubs_raw['clubs'])

In [ ]:
eng_clubs.head()
#the JSONs are now normalized.

,name,code,country
0,Watford FC,WAT,England
1,AFC Bournemouth,BOU,England
2,Norwich City FC,NOR,England
3,Burnley FC,BUR,England
4,Manchester United FC,MUN,England


**Using epl dataframe as main example, Here are the basic data exploration from my datasets.**

In [ ]:
epl.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,09/08/2019,20:00,Liverpool,Norwich,4,1,H,4,0,...,3.43,-2.25,1.91,1.99,1.94,1.98,1.99,2.07,1.90,1.99
1,E0,10/08/2019,12:30,West Ham,Man City,0,5,A,0,1,...,2.91,1.75,1.95,1.95,1.96,1.97,2.07,1.98,1.97,1.92
2,E0,10/08/2019,15:00,Bournemouth,Sheffield United,1,1,D,0,0,...,1.92,-0.50,1.95,1.95,1.98,1.95,2.00,1.96,1.96,1.92
3,E0,10/08/2019,15:00,Burnley,Southampton,3,0,H,0,0,...,1.71,0.00,1.87,2.03,1.89,2.03,1.90,2.07,1.86,2.02
4,E0,10/08/2019,15:00,Crystal Palace,Everton,0,0,D,0,0,...,1.71,0.25,1.82,2.08,1.97,1.96,2.03,2.08,1.96,1.93


In [ ]:
epl.dtypes

,0
Div,object
Date,object
Time,object
HomeTeam,object
AwayTeam,object
...,...
PCAHA,float64
MaxCAHH,float64
MaxCAHA,float64
AvgCAHH,float64


In [ ]:
epl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Columns: 106 entries, Div to AvgCAHA
dtypes: float64(82), int64(16), object(8)
memory usage: 172.4+ KB


In [ ]:
epl

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,09/08/2019,20:00,Liverpool,Norwich,4,1,H,4,0,...,3.43,-2.25,1.91,1.99,1.94,1.98,1.99,2.07,1.90,1.99
1,E0,10/08/2019,12:30,West Ham,Man City,0,5,A,0,1,...,2.91,1.75,1.95,1.95,1.96,1.97,2.07,1.98,1.97,1.92
2,E0,10/08/2019,15:00,Bournemouth,Sheffield United,1,1,D,0,0,...,1.92,-0.50,1.95,1.95,1.98,1.95,2.00,1.96,1.96,1.92
3,E0,10/08/2019,15:00,Burnley,Southampton,3,0,H,0,0,...,1.71,0.00,1.87,2.03,1.89,2.03,1.90,2.07,1.86,2.02
4,E0,10/08/2019,15:00,Crystal Palace,Everton,0,0,D,0,0,...,1.71,0.25,1.82,2.08,1.97,1.96,2.03,2.08,1.96,1.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,E0,01/01/2020,15:00,Watford,Wolves,2,1,H,1,0,...,1.68,0.25,1.82,2.11,1.83,2.09,1.90,2.17,1.83,2.05
204,E0,01/01/2020,17:30,Man City,Everton,2,1,H,0,0,...,2.76,-1.50,1.88,2.05,1.92,2.01,1.95,2.14,1.88,2.00
205,E0,01/01/2020,17:30,Norwich,Crystal Palace,1,1,D,1,0,...,1.76,-0.25,2.08,1.85,2.05,1.88,2.19,1.92,2.04,1.84
206,E0,01/01/2020,17:30,West Ham,Bournemouth,4,0,H,3,0,...,1.94,-0.50,1.97,1.96,1.97,1.96,2.00,2.00,1.93,1.94


#Data Transformation:

- **To check the team names in my files and check if they are uniform or not.**

In [ ]:
# Check unique team names in the match data
print("Unique HomeTeam names in match data:\n", epl['HomeTeam'].unique())

print("Unique HomeTeam names in match data:\n", laliga['HomeTeam'].unique())

print("Unique HomeTeam names in match data:\n", bundesliga['HomeTeam'].unique())

print("Unique HomeTeam names in match data:\n", ligue_1['HomeTeam'].unique())

print("Unique HomeTeam names in match data:\n", serie_a['HomeTeam'].unique())

# Check unique names in the club data
print("Unique names in club data:\n", eng_clubs['name'].unique())

print("Unique names in club data:\n", spa_clubs['name'].unique())

print("Unique names in club data:\n", ger_clubs['name'].unique())

print("Unique names in club data:\n", fre_clubs['name'].unique())

print("Unique names in club data:\n", ita_clubs['name'].unique())




Unique HomeTeam names in match data:
 ['Liverpool' 'West Ham' 'Bournemouth' 'Burnley' 'Crystal Palace' 'Watford'
 'Tottenham' 'Leicester' 'Newcastle' 'Man United' 'Arsenal' 'Aston Villa'
 'Brighton' 'Everton' 'Norwich' 'Southampton' 'Man City'
 'Sheffield United' 'Chelsea' 'Wolves']
Unique HomeTeam names in match data:
 ['Ath Bilbao' 'Celta' 'Valencia' 'Mallorca' 'Leganes' 'Villarreal'
 'Alaves' 'Espanol' 'Betis' 'Ath Madrid' 'Granada' 'Levante' 'Osasuna'
 'Real Madrid' 'Getafe' 'Barcelona' 'Sevilla' 'Sociedad' 'Eibar'
 'Valladolid']
Unique HomeTeam names in match data:
 ['Bayern Munich' 'Dortmund' 'Freiburg' 'Leverkusen' 'Werder Bremen'
 'Wolfsburg' "M'gladbach" 'Ein Frankfurt' 'Union Berlin' 'FC Koln'
 'Augsburg' 'Fortuna Dusseldorf' 'Hoffenheim' 'Mainz' 'Paderborn'
 'Schalke 04' 'RB Leipzig' 'Hertha']
Unique HomeTeam names in match data:
 ['Monaco' 'Marseille' 'Angers' 'Brest' 'Dijon' 'Montpellier' 'Nice'
 'Lille' 'Strasbourg' 'Paris SG' 'Lyon' 'Nantes' 'Amiens' 'Bordeaux'
 'Metz' '

- **Since the names in the csv and JSONs are different, Below are the mapings so the names are similar in their respective data files.**

In [ ]:
epl_mapping = {
    "Liverpool": "Liverpool FC",
    "West Ham": "West Ham United FC",
    "Bournemouth": "AFC Bournemouth",
    "Burnley": "Burnley FC",
    "Crystal Palace": "Crystal Palace FC",
    "Watford": "Watford FC",
    "Tottenham": "Tottenham Hotspur FC",
    "Leicester": "Leicester City FC",
    "Newcastle": "Newcastle United FC",
    "Man United": "Manchester United FC",
    "Man City": "Manchester City FC",
    "Arsenal": "Arsenal FC",
    "Aston Villa": "Aston Villa FC",
    "Brighton": "Brighton & Hove Albion FC",
    "Everton": "Everton FC",
    "Norwich": "Norwich City FC",
    "Southampton": "Southampton FC",
    "Sheffield United": "Sheffield United FC",
    "Chelsea": "Chelsea FC",
    "Wolves": "Wolverhampton Wanderers FC"
}

In [ ]:
laliga_mapping = {
    "Ath Bilbao": "Athletic Club Bilbao",
    "Celta": "RC Celta Vigo",
    "Valencia": "Valencia CF",
    "Mallorca": "RCD Mallorca",
    "Leganes": "CD Leganés",
    "Villarreal": "Villarreal CF",
    "Alaves": "Deportivo Alavés",
    "Espanol": "RCD Espanyol",
    "Betis": "Real Betis",
    "Ath Madrid": "Atlético Madrid",
    "Granada": "Granada CF",
    "Levante": "Levante UD",
    "Osasuna": "CA Osasuna",
    "Real Madrid": "Real Madrid",
    "Getafe": "Getafe CF",
    "Barcelona": "FC Barcelona",
    "Sevilla": "Sevilla FC",
    "Sociedad": "Real Sociedad",
    "Eibar": "SD Eibar",
    "Valladolid": "Real Valladolid CF"
}

In [ ]:
bundesliga_mapping = {
    "Bayern Munich": "Bayern München",
    "Dortmund": "Borussia Dortmund",
    "Freiburg": "SC Freiburg",
    "Leverkusen": "Bayer 04 Leverkusen",
    "Werder Bremen": "Werder Bremen",
    "Wolfsburg": "VfL Wolfsburg",
    "M'gladbach": "Bor. Mönchengladbach",
    "Ein Frankfurt": "Eintracht Frankfurt",
    "Union Berlin": "1. FC Union Berlin",
    "FC Koln": "1. FC Köln",
    "Augsburg": "FC Augsburg",
    "Fortuna Dusseldorf": "Fortuna Düsseldorf",
    "Hoffenheim": "TSG 1899 Hoffenheim",
    "Mainz": "1. FSV Mainz 05",
    "Paderborn": "SC Paderborn 07",
    "Schalke 04": "FC Schalke 04",
    "RB Leipzig": "RB Leipzig",
    "Hertha": "Hertha BSC"
}

In [ ]:
ligue_1_mapping = {
    "Monaco": "AS Monaco",
    "Marseille": "Olympique de Marseille",
    "Angers": "Angers SCO",
    "Brest": "Stade Brestois 29",
    "Dijon": "Dijon FCO",
    "Montpellier": "Montpellier HSC",
    "Nice": "OGC Nice",
    "Lille": "Lille OSC",
    "Strasbourg": "RC Strasbourg",
    "Paris SG": "Paris Saint-Germain",
    "Lyon": "Olympique Lyonnais",
    "Nantes": "FC Nantes",
    "Amiens": "Amiens SC",
    "Bordeaux": "Girondins de Bordeaux",
    "Metz": "FC Metz",
    "Nimes": "Nîmes Olympique",
    "Toulouse": "Toulouse FC",
    "St Etienne": "AS Saint-Étienne",
    "Reims": "Stade de Reims",
    "Rennes": "Stade Rennais FC"
}

In [ ]:
serie_a_mapping = {
    "Parma": "Parma",
    "Fiorentina": "ACF Fiorentina",
    "Udinese": "Udinese Calcio",
    "Cagliari": "Cagliari Calcio",
    "Roma": "AS Roma",
    "Sampdoria": "UC Sampdoria",
    "Spal": "SPAL Ferrara",
    "Torino": "Torino FC",
    "Verona": "Hellas Verona",
    "Inter": "FC Internazionale Milano",
    "Bologna": "Bologna FC",
    "Milan": "AC Milan",
    "Juventus": "Juventus",
    "Lazio": "SS Lazio",
    "Atalanta": "Atalanta Bergamo",
    "Genoa": "Genoa CFC",
    "Lecce": "US Lecce",
    "Sassuolo": "US Sassuolo Calcio",
    "Napoli": "SSC Napoli",
    "Brescia": "Brescia Calcio"
}

- **Replacing the teams names with the help of above mappings in my dataframe**

In [ ]:
# Apply mappings to EPL
epl['HomeTeam'] = epl['HomeTeam'].replace(epl_mapping)
epl['AwayTeam'] = epl['AwayTeam'].replace(epl_mapping)

# Apply mappings to La Liga
laliga['HomeTeam'] = laliga['HomeTeam'].replace(laliga_mapping)
laliga['AwayTeam'] = laliga['AwayTeam'].replace(laliga_mapping)

# Apply mappings to Bundesliga
bundesliga['HomeTeam'] = bundesliga['HomeTeam'].replace(bundesliga_mapping)
bundesliga['AwayTeam'] = bundesliga['AwayTeam'].replace(bundesliga_mapping)

# Apply mappings to Ligue 1
ligue_1['HomeTeam'] = ligue_1['HomeTeam'].replace(ligue_1_mapping)
ligue_1['AwayTeam'] = ligue_1['AwayTeam'].replace(ligue_1_mapping)

# Apply mappings to Serie A
serie_a['HomeTeam'] = serie_a['HomeTeam'].replace(serie_a_mapping)
serie_a['AwayTeam'] = serie_a['AwayTeam'].replace(serie_a_mapping)


- **Merging the datafiles with their respective leagues based on team names and codes.**
- **Also doing some transformations like dropping and renaming few columns.**

In [ ]:
# Merge EPL match data with the club data to add team codes
epl = epl.merge(eng_clubs[['name', 'code']], left_on='HomeTeam', right_on='name', how='left', suffixes=('', '_home'))
epl = epl.merge(eng_clubs[['name', 'code']], left_on='AwayTeam', right_on='name', how='left', suffixes=('', '_away'))

epl = epl.drop(columns=['name', 'name_away'])

epl = epl.rename(columns={'code': 'HomeTeamCode', 'code_away': 'AwayTeamCode'})

print(epl[['HomeTeam', 'HomeTeamCode', 'AwayTeam', 'AwayTeamCode']].head())


             HomeTeam HomeTeamCode             AwayTeam AwayTeamCode
0        Liverpool FC          LIV      Norwich City FC          NOR
1  West Ham United FC          WHU   Manchester City FC          MCI
2     AFC Bournemouth          BOU  Sheffield United FC          SHU
3          Burnley FC          BUR       Southampton FC          SOU
4   Crystal Palace FC          CRY           Everton FC          EVE


In [ ]:
# Merge La Liga match data with the club data to add team codes
laliga = laliga.merge(spa_clubs[['name', 'code']], left_on='HomeTeam', right_on='name', how='left', suffixes=('', '_home'))
laliga = laliga.merge(spa_clubs[['name', 'code']], left_on='AwayTeam', right_on='name', how='left', suffixes=('', '_away'))

laliga = laliga.drop(columns=['name', 'name_away'])

laliga = laliga.rename(columns={'code': 'HomeTeamCode', 'code_away': 'AwayTeamCode'})

print(laliga[['HomeTeam', 'HomeTeamCode', 'AwayTeam', 'AwayTeamCode']].head())

               HomeTeam HomeTeamCode       AwayTeam AwayTeamCode
0  Athletic Club Bilbao          ATH   FC Barcelona          BAR
1         RC Celta Vigo          CEL    Real Madrid          RMD
2           Valencia CF          VAL  Real Sociedad          RSO
3          RCD Mallorca          MLL       SD Eibar         None
4            CD Leganés         None     CA Osasuna          OSA


In [ ]:
# Merge Bundesliga match data with the club data to add team codes
bundesliga = bundesliga.merge(ger_clubs[['name', 'code']], left_on='HomeTeam', right_on='name', how='left', suffixes=('', '_home'))
bundesliga = bundesliga.merge(ger_clubs[['name', 'code']], left_on='AwayTeam', right_on='name', how='left', suffixes=('', '_away'))

bundesliga = bundesliga.drop(columns=['name', 'name_away'])

bundesliga = bundesliga.rename(columns={'code': 'HomeTeamCode', 'code_away': 'AwayTeamCode'})

print(bundesliga[['HomeTeam', 'HomeTeamCode', 'AwayTeam', 'AwayTeamCode']].head())


              HomeTeam HomeTeamCode            AwayTeam AwayTeamCode
0       Bayern München          FCB          Hertha BSC          BSC
1    Borussia Dortmund          BVB         FC Augsburg          FCA
2          SC Freiburg          SCF     1. FSV Mainz 05          M05
3  Bayer 04 Leverkusen          B04     SC Paderborn 07          SCP
4        Werder Bremen          BRE  Fortuna Düsseldorf          F95


In [ ]:
# Merge Serie A match data with the club data to add team codes
serie_a = serie_a.merge(ita_clubs[['name', 'code']], left_on='HomeTeam', right_on='name', how='left', suffixes=('', '_home'))
serie_a = serie_a.merge(ita_clubs[['name', 'code']], left_on='AwayTeam', right_on='name', how='left', suffixes=('', '_away'))

serie_a = serie_a.drop(columns=['name', 'name_away'])

serie_a = serie_a.rename(columns={'code': 'HomeTeamCode', 'code_away': 'AwayTeamCode'})

print(serie_a[['HomeTeam', 'HomeTeamCode', 'AwayTeam', 'AwayTeamCode']].head())


          HomeTeam HomeTeamCode        AwayTeam AwayTeamCode
0            Parma          PAR        Juventus          JUV
1   ACF Fiorentina          FIO      SSC Napoli          NAP
2   Udinese Calcio          UDI        AC Milan          MIL
3  Cagliari Calcio          CAG  Brescia Calcio         None
4          AS Roma          ROM       Genoa CFC          GEN


In [ ]:
# Merge Ligue 1 match data with the club data to add team codes
ligue_1 = ligue_1.merge(fre_clubs[['name', 'code']], left_on='HomeTeam', right_on='name', how='left', suffixes=('', '_home'))
ligue_1 = ligue_1.merge(fre_clubs[['name', 'code']], left_on='AwayTeam', right_on='name', how='left', suffixes=('', '_away'))

ligue_1 = ligue_1.drop(columns=['name', 'name_away'])

ligue_1 = ligue_1.rename(columns={'code': 'HomeTeamCode', 'code_away': 'AwayTeamCode'})

print(ligue_1[['HomeTeam', 'HomeTeamCode', 'AwayTeam', 'AwayTeamCode']].head())


                 HomeTeam HomeTeamCode               AwayTeam AwayTeamCode
0               AS Monaco          ASM     Olympique Lyonnais           OL
1  Olympique de Marseille           OM         Stade de Reims          SDR
2              Angers SCO          SCO  Girondins de Bordeaux         FCGB
3       Stade Brestois 29         SB29            Toulouse FC          TFC
4               Dijon FCO         DFCO       AS Saint-Étienne         ASSE


- **Converting the 'Date' column from object to datetime**

In [ ]:
epl['Date'] = pd.to_datetime(epl['Date'], format='%d/%m/%Y')

laliga['Date'] = pd.to_datetime(laliga['Date'], format='%d/%m/%Y')

bundesliga['Date'] = pd.to_datetime(bundesliga['Date'], format='%d/%m/%Y')

serie_a['Date'] = pd.to_datetime(serie_a['Date'], format='%d/%m/%Y')

ligue_1['Date'] = pd.to_datetime(ligue_1['Date'], format='%d/%m/%Y')


- *You can see below that the datatype has been changed for Date column from our initial data exploration*

In [ ]:
epl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Columns: 108 entries, Div to AwayTeamCode
dtypes: datetime64[ns](1), float64(82), int64(16), object(9)
memory usage: 175.6+ KB


In [ ]:
epl.dtypes

,0
Div,object
Date,datetime64[ns]
Time,object
HomeTeam,object
AwayTeam,object
...,...
MaxCAHA,float64
AvgCAHH,float64
AvgCAHA,float64
HomeTeamCode,object


- **Adding a Total Goals column to the dataframes to show the no. of goals scored in each match**

In [ ]:
epl['TotalGoals'] = epl['FTHG'] + epl['FTAG']

laliga['TotalGoals'] = laliga['FTHG'] + laliga['FTAG']

bundesliga['TotalGoals'] = bundesliga['FTHG'] + bundesliga['FTAG']

serie_a['TotalGoals'] = serie_a['FTHG'] + serie_a['FTAG']

ligue_1['TotalGoals'] = ligue_1['FTHG'] + ligue_1['FTAG']


In [ ]:
epl.head()
# to show the newly added column into the df

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,HomeTeamCode,AwayTeamCode,TotalGoals
0,E0,2019-08-09,20:00,Liverpool FC,Norwich City FC,4,1,H,4,0,...,1.99,1.94,1.98,1.99,2.07,1.90,1.99,LIV,NOR,5
1,E0,2019-08-10,12:30,West Ham United FC,Manchester City FC,0,5,A,0,1,...,1.95,1.96,1.97,2.07,1.98,1.97,1.92,WHU,MCI,5
2,E0,2019-08-10,15:00,AFC Bournemouth,Sheffield United FC,1,1,D,0,0,...,1.95,1.98,1.95,2.00,1.96,1.96,1.92,BOU,SHU,2
3,E0,2019-08-10,15:00,Burnley FC,Southampton FC,3,0,H,0,0,...,2.03,1.89,2.03,1.90,2.07,1.86,2.02,BUR,SOU,3
4,E0,2019-08-10,15:00,Crystal Palace FC,Everton FC,0,0,D,0,0,...,2.08,1.97,1.96,2.03,2.08,1.96,1.93,CRY,EVE,0


In [ ]:
bundesliga.head()
#same as above but with different league

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,D1,16/08/2019,19:30,Bayern Munich,Hertha,2,2,D,1,2,...,3.40,-2.25,2.03,1.90,1.99,1.93,2.04,1.93,1.98,1.91
1,D1,17/08/2019,14:30,Dortmund,Augsburg,5,1,H,1,1,...,3.31,-2.25,1.92,2.01,1.92,2.00,1.98,2.04,1.91,1.97
2,D1,17/08/2019,14:30,Freiburg,Mainz,3,0,H,0,0,...,1.89,0.00,1.92,2.01,1.94,1.97,1.97,2.06,1.90,1.99
3,D1,17/08/2019,14:30,Leverkusen,Paderborn,3,2,H,2,2,...,3.58,-2.00,2.07,1.86,2.05,1.86,2.15,1.91,2.03,1.85
4,D1,17/08/2019,14:30,Werder Bremen,Fortuna Dusseldorf,1,3,A,0,1,...,2.26,-0.75,1.92,2.01,1.92,2.00,1.95,2.11,1.89,2.00
